
```
 _____                        _
|_   _|                      (_)
  | | ___ _ __ _ __ ___  _ __ _  __ _ 
  | |/ _ \ '__| '__/ _ \| '__| |/ _` |
  | |  __/ |  | | | (_) | |  | | (_| |
  \_/\___|_|  |_|  \___/|_|  |_|\__,_|

```
**A Terraria server to deploy on Google Colab**

---

This code needs two simple scripts. Running them in order will be enough for you to have a Terraria server running on Google Colab. Take into account that you might want to adjust the regions where the server is hosted. At the moment, the only supported service for deployment is ngrok. Clear instructions will be on the readme file.

This is the first script. It will install the necessary dependencies and download the server files to your drive folder. It will also create the terraria session needed.


In [ ]:
import os
import zipfile
import shutil
import requests
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to the directory containing the Terraria server
server_dir = '/content/drive/My Drive/terraria-server'
if not os.path.exists(server_dir):
    os.makedirs(server_dir)
%cd $server_dir

# Download Terraria server files
SERVER_URL = 'https://terraria.org/api/download/pc-dedicated-server/terraria-server-1449.zip'
response = requests.get(SERVER_URL)

# Downloads the file from the URL
with open('terraria-server-1449.zip', 'wb') as file:
    file.write(response.content)

# Extract the files
with zipfile.ZipFile('terraria-server-1449.zip', 'r') as zip_ref:
    zip_ref.extractall('terraria-server-1449')

# Navigate into the extracted directory
os.chdir('terraria-server-1449/1449/Linux')  # Corrected working directory path

# Add executable permissions to the server binary
!chmod +x TerrariaServer.bin.x86_64

# Move the server files to a folder in your Google Drive
server_files_dir = '/content/drive/My Drive/terraria-server/terraria-server-1449'
server_executable = 'TerrariaServer.bin.x86_64'
destination_path = os.path.join(server_files_dir, server_executable)

if not os.path.exists(destination_path):
    shutil.move(server_executable, server_files_dir)

#Install tmux and create the session
!apt-get -qq install tmux
!tmux new-session -d -s terraria './TerrariaServer.bin.x86_64 -port {server_port}' # Will create if not exists


# Run the server

This is the second script. It will run the server and create a tunnel to it using ngrok. You will need to create an account on ngrok and get your authtoken. You can find it on your ngrok dashboard.


In [ ]:
!pip install pyngrok
import os
from pyngrok import conf, ngrok

# Navigate to the directory containing the Terraria server (if not already there)
server_dir = '/content/drive/My Drive/terraria-server/terraria-server-1449/1449/Linux'
os.chdir(server_dir)
!chmod +x TerrariaServer.bin.x86_64

# Add executable permissions to the server binary (if not already done)
server_executable = 'TerrariaServer.bin.x86_64'
server_executable_path = os.path.join(server_dir, server_executable)
if not os.access(server_executable_path, os.X_OK):
    !chmod +x {server_executable_path}

# Authenticate ngrok with your authtoken
authtoken = input('Enter your ngrok authtoken: ')  # Input your ngrok authtoken here
!ngrok authtoken $authtoken

# Sets default ngrok region
conf.get_default().region = 'us'  # Change this to whichever region you want
url = ngrok.connect(7777, 'tcp')
print('This is your url:', url)

# Launch the Terraria server using tmux (in the same cell)
!tmux list-sessions
!tmux attach-session -t terraria


# Special thanks
Thanks to [thecoder-001](https://github.com/thecoder-001). They created the code for this to be run on Minecraft. I inspired a big part of their code to make mine into reality, and it served as a guide for me to understand how to make this work.
